In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-05-01"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
12125,2024-05-01,Israel Liga Leumit,12:45,Maccabi Rehovot,Elitzur Ashkelon,1.36,2.97,157.5,1.87,1.79,0.0,0.00,0.00,MHT1dfOi,0.735294,0.336700,0.534759,0.558659,0.071994,126.106,21.856513,0.173319,1.8,1.643168,0.912871,157.70,1.664,0.245927,0.147793,9.0,132.524,45.241048,0.341380,0.0,0.000000,NaN,141.96,1.844,0.336422,0.182442,-47.0,83,78,1.90,1.82,115.494,135.224,0.525839,0.030912,NaN,-1.09,-0.218,-1.651376,0.715229,0.8,0.084771,-5.00,-1.000,-1.970000,0.000000,0.0,0.000000
12126,2024-05-01,Lituânia Lkl,12:00,Pieno Zvaigzdes,Jonava,1.95,1.77,170.5,1.82,1.88,1.5,1.82,1.64,O0Uzyrti,0.512821,0.564972,0.549451,0.531915,0.077792,497.388,228.593087,0.459587,0.6,1.341641,2.236068,635.36,6.260,2.870845,0.458601,-63.0,217.040,112.325671,0.517534,1.2,1.643168,1.369306,404.60,2.656,1.294230,0.487285,-31.0,88,85,7.22,4.76,277.380,369.872,0.068430,0.022933,0.073572,-5.00,-1.000,-0.950000,0.000000,0.0,0.000000,7.63,1.526,0.504587,0.382322,0.2,-0.182322
12127,2024-05-01,Kosovo Superliga,14:00,Golden Eagle Ylli,Peja,1.51,2.45,167.5,1.85,1.85,0.0,0.00,0.00,2F1Qbk7F,0.662252,0.408163,0.540541,0.540541,0.070415,151.724,31.269976,0.206098,2.4,1.341641,0.559017,166.84,1.782,0.490275,0.275127,46.0,143.068,23.181586,0.162032,1.2,1.643168,1.369306,169.54,1.672,0.166343,0.099487,4.0,86,98,1.94,1.73,0.000,0.000,0.335697,0.000000,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
12128,2024-05-01,Itália Série A1 Feminina,15:00,Campobasso F,San Giovanni F,1.67,2.12,124.5,1.87,1.83,0.0,0.00,0.00,nVV20J0f,0.598802,0.471698,0.534759,0.546448,0.070501,135.608,84.521868,0.623281,2.4,1.341641,0.559017,144.30,2.100,1.523220,0.725343,35.0,93.868,25.044391,0.266804,1.8,1.643168,0.912871,99.82,1.556,0.708329,0.455224,12.0,65,62,2.22,1.61,132.292,126.192,0.167915,0.015289,NaN,0.87,0.174,3.850575,0.000000,0.0,0.000000,1.04,0.208,5.384615,0.000000,0.0,0.000000
12129,2024-05-01,Itália Série A1 Feminina,13:00,Virtus Bologna F,Ragusa F,1.12,5.51,135.5,1.83,1.83,0.0,0.00,0.00,EHXf2uWs,0.892857,0.181488,0.546448,0.546448,0.074345,96.336,24.039992,0.249543,3.0,0.000000,0.000000,108.54,1.416,0.400100,0.282556,64.0,156.856,84.034577,0.535743,0.6,1.341641,2.236068,181.72,2.514,1.561419,0.621090,-36.0,81,59,1.34,3.08,87.502,128.366,0.936410,0.000000,NaN,-0.64,-0.128,-0.937500,0.000000,0.0,0.000000,-0.99,-0.198,-22.777778,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12201,2024-05-01,Bulgária Nbl,13:00,Cherno More,Spartak Pleven,4.13,1.21,173.5,1.86,1.84,0.0,0.00,0.00,294moSI6,0.242131,0.826446,0.537634,0.543478,0.068577,549.858,159.438513,0.289963,0.0,0.000000,NaN,701.35,7.518,2.169071,0.288517,-101.0,145.678,26.058803,0.178879,2.4,1.341641,0.559017,121.80,1.718,0.639508,0.372240,69.0,83,116,8.45,1.05,249.788,196.844,0.773315,0.007644,NaN,-1.75,-0.350,-8.942857,0.399945,0.5,0.100055,-2.06,-0.412,-0.509709,0.528399,0.5,-0.028399
12202,2024-05-01,Bulgária Nbl,12:00,Shumen,Rilski Sportist,8.25,1.05,168.5,1.81,1.85,0.0,0.00,0.00,Wn1OnufB,0.121212,0.952381,0.552486,0.540541,0.073593,202.782,116.826109,0.576117,1.8,1.643168,0.

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

Sem jogos
